## Scraper Summarizer Upgrade

Upgrading the project to summarize a webpage and use an Open Source model running locally via Ollama rather than OpenAI

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model


### Installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  [http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that used for OpenAI (list of dictionaries)

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [7]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Installation of Ollama' at the top of this notebook

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI (GAI) has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: GAI can generate high-quality content, such as articles, social media posts, product descriptions, and even entire books. This can help businesses save time and resources on content creation, while also providing unique and personalized content for their audience.
2. **Product Design and Development**: Generative AI can be used to design new products, including 3D models, prototypes, and even fully functional products. This can accelerate the product development process, reduce costs, and improve product quality.
3. **Marketing and Advertising**: GAI can generate personalized marketing campaigns, ads, and promotional materials that cater to individual customers' interests and preferences. This can help businesses increase engagement, conversion rates, and overall ROI.
4. **Customer Service and Support**: Generative AI-powered chatbots and virtual assistant

## Introducing the *Ollama* package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [9]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and resources on content creation.
2. **Product Design and Development**: Generative AI can assist in designing new products by generating 3D models, prototypes, and even entire product lines. This can help companies reduce design costs and accelerate product development.
3. **Marketing and Advertising**: Generative AI can be used to generate personalized ads, social media posts, and email campaigns that resonate with target audiences. It can also help analyze customer behavior and preferences.
4. **Customer Service and Support**: Generative AI-powered chatbots can provide 24/7 customer support, helping to reduce response times and improve customer satisfaction.
5. **Data Analysis and Visualization**:

### Alternative approach - using *OpenAI python library* to connect to *Ollama*

In [10]:
# An alternative approach that some people might prefer which is using the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI, a subset of artificial intelligence (AI) that enables machines to create new, original data, has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as blog posts, product descriptions, social media posts, and even entire books. This can help reduce content creation costs and improve productivity.
2. **Image and Video Editing**: Generative AI algorithms can edit images and videos to enhance their quality or create new ones from scratch. This can be applied to various fields like advertising, entertainment, and e-commerce.
3. **Recommendation Systems**: Generative AI-powered recommendation systems can suggest products or services based on user behavior, preferences, and patterns. This is commonly used in e-commerce, streaming services, and advertising.
4. **Chatbots and Conversational Interfaces**: Generative AI enables the development of intelligent c

### QQ: Are you confused about why that works?

Strange? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

### Also trying the amazing reasoning model *DeepSeek*

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [11]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕                ▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕       

In [12]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Alright, so I need to figure out how to answer the questions about Neural Networks, Attention, and Transformers in the context of Large Language Models (LLMs). Let me break this down step by step.

First, I should start by understanding what a Neural Network is. From the initial response, it was defined as a computational model inspired by the structure and function of biological neural networks. It's composed of layers that process input data. The layers include an encoder layer for capturing sequence-level representations and a decoder layer to generate outputs. Each layer applies weights and biases through non-linear activation functions like ReLU or the sigmoid function, enabling learning and adaptation. So, I need to elaborate on how exactly neural networks work, their structure, types (like convolutional vs recurrent), training methods, layers, hidden nodes, backpropagation, optimization algorithms, differentarchitectures, benefits, limitations.

Next, attention is mentio

### NOW let's upgrade the *Scraper Summarizer* project leveraging a local model through *Ollama*!

Basically the goal is to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; using either of the above approaches.

In [13]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [14]:
# Define our system prompt - you can experiment with this i.e. changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
# This function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]
    
def summarize(url):
    website = Website(url)
    # response = openai.chat.completions.create(
    #     model = "gpt-4o-mini",
    #     messages = messages_for(website)
    # )
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']

# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://foxnews.com")

This appears to be a sample Fox News website, likely from 2025 or later. I'll provide an overview of the content and features.

**Content**

The website features a mix of news articles, videos, podcasts, and other digital content. The headlines are divided into several sections:

1. **U.S.**
2. **Crime**
3. **Terror**
4. **True Crime**
5. **Bryan Kohberger**
6. **Karen Read**
7. **Politics**
8. **Trump's First 100 Days**
9. **Senate**
10. **House**
11. **Judiciary**
12. **Foreign Policy**
13. **Fox News Polls**
14. **Elections**
15. **World**
16. **U.N.**
17. **Conflicts**
18. **Terrorism**
19. **Disasters**
20. **Global Economy**
21. **Environment**
22. **Religion**
23. **Scandals**
24. **Opinion**

**Features**

1. **FOX News Shows**: A list of current and upcoming shows, including programming schedule and on-air personalities.
2. **Programming Schedule**: A detailed schedule of Fox News programs and podcasts.
3. **On Air Personalities**: Profiles of the hosts and guests featured on various shows.
4. **Show Clips**: Short clips from recent episodes of Fox News programs.
5. **News Clips**: Brief news summaries from around the world.
6. **FOX Nation**: A section dedicated to conservative culture, lifestyle, and entertainment.
7. **FOX Noticias**: Spanish-language news content and videos.
8. **FOX News Shop**: An e-commerce platform for purchasing Fox-branded merchandise.
9. **FOX News Radio**: A list of affiliate radio stations and podcast episodes.

**Design**

The website's design is modern and minimalist, with a predominantly white and blue color scheme. The layout is responsive, adapting to various screen sizes. The font used is clear and easy to read, making the content accessible on multiple devices.

**Social Media Links**

Fox News has prominent social media links at the bottom of the page, including:

1. Facebook
2. Twitter
3. Instagram
4. YouTube

Overall, this Fox News website provides a comprehensive platform for readers to stay up-to-date on news, politics, and current events from a conservative perspective.